In [ ]:
import numpy as np
import tensorflow as tf
import scipy.io
from keras import backend as K
from keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects

In [ ]:
# define the filename to save the weights of the model as a .mat
filename = 'models/sin_tri_stable'

# define the interval of the domain for the visible dynamics
lower_limit = 1
upper_limit = 18
step_size = 0.0001

# define the offset parameter in the smooth ReLU
gamma = 1

# define the feedback offset parameter in the feedback motif
beta_1 = 0

# define the number of additional species in a single layer
N = 4

# define the number of epochs to use in training the model
number_of_epochs = 2

In [ ]:
# define the custom activation function for the training with particular values of the relu_offset
def smooth_max_activation(x):
    return 0.5*(x + K.sqrt(K.square(x)+4*gamma))

get_custom_objects().update({'smooth_max_activation': Activation(smooth_max_activation)})

# initialize the range of visible molecular concentrations over the desired interval
x_train = np.arange(lower_limit, upper_limit, step_size, dtype="float32")

# define the desired function of the visible dynamics
f_x = np.sin(x_train)

# map the desired function of the visible dynamics onto the function learnt by the neural subsystem
y_train = (f_x - beta_1) /x_train

# define a neural network model that corresponds to the asymptotic neural subsytem 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1,), name=''),
  tf.keras.layers.Dense(N, activation='smooth_max_activation'),
  tf.keras.layers.Dense(1, activation=None, use_bias=False),
])

# compile model with optimizer and choice of loss function
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

In [ ]:
# train the neural network model to replicate x_train -> y_train
model.fit(x_train, y_train, epochs=number_of_epochs, validation_split = 0.1)

In [ ]:
# view the model structure, each 'Param' corresponds to at least one rate of reaction in the chemical system
model.summary()

# save the weights of this neural network for use in ODE simulations in MATLAB
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases = model.layers[1].get_weights()[1]
output_layer_weights = model.layers[2].get_weights()[0]


In [ ]:
scipy.io.savemat(filename+'.mat', {'first_layer_weights':first_layer_weights, 'first_layer_biases':first_layer_biases, 'output_layer_weights':output_layer_weights})